<a href="https://colab.research.google.com/github/jpscard/IA-BIO-INSPIRADA-E-OTIMIZACAO/blob/main/Exerc%C3%ADcio_3_Problemas_de_Ger%C3%AAncia_de_Energia_com_pre%C3%A7o_(BLP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Exercício 1 - Problemas de Programação Linear

### Professores:
 - Profª Drª Jasmine Priscyla Leite de Araújo
 - Prof. Dr. Glauco Estácio Gonçalves

### Alunos:
 - João Paulo da Silva Cardoso
 - Luiz Paulo Coutinho de Souza

# Problema de Gerenciamento de Energia por Desligamento de Servidores

Neste problema, faremos o gerenciamento de energia por meio do desligamento de servidores. Temos um conjunto \(\{1 \ldots V\}\) de VMs, sendo que cada VM \(i\) possui apenas a sua respectiva demanda de capacidade \(d_i\). Além disso, temos um conjunto de servidores \(\{1 \ldots M\}\), com cada servidor \(j\) possuindo sua respectiva capacidade \(C_j\). Um servidor pode ou não estar ligado, o que é modelado por meio da variável de decisão booleana \(y_j\), com 1 representando o servidor ligado e 0 desligado. A alocação de uma VM \(i\) em um servidor \(j\) é modelada pela variável de decisão booleana \(x_{ij}\) que assume 1 no caso da máquina virtual \(i\) estar alocada no servidor \(j\), e 0 caso contrário.

Objetivo: Minimizar o número de servidores ativos.
$$
\text{minimizar } \sum_{j=1}^{M} y_j
$$

Sujeito às seguintes restrições:
$$
\sum_{i=1}^{V} d_i x_{ij} \leq y_j C_j, \text{ para cada servidor } j
$$
$$
\sum_{j=1}^{M} x_{ij} = 1, \text{ para cada VM } i
$$
$$
x_{ij} \in \{0,1\}, \text{ para todo } i \text{ e } j
$$
$$
y_j \in \{0,1\}, \text{ para todo } j
$$

Este problema é NP-difícil e equivale ao problema conhecido como Bin-packing. Diversas formas de solucioná-lo são apresentadas no Capítulo 8 em [1].

## Adicionando Preço (BLP)

Podemos modificar este problema adicionando preço. Para tanto, basta que adicionemos a variável do preço \(p_i\) de cada VM \(i\) e um custo energético \(w_j\) de cada servidor. Assim, desejamos otimizar a seguinte função objetivo:

Objetivo: Maximizar
$$
\text{maximizar } \sum_{i=1}^{V} \sum_{j=1}^{M} p_i x_{ij} - \sum_{j=1}^{M} w_j y_j
$$

Sujeita às seguintes restrições:

1. Para cada servidor \(j\):
   $$
   \sum_{i=1}^{V} d_i x_{ij} \leq y_j C_j
   $$

2. Para cada VM \(i\):
   $$
   \sum_{j=1}^{M} x_{ij} \leq 1
   $$

3. Para todo \(i\) e \(j\):
   $$
   x_{ij} \in \{0,1\}
   $$
   $$
   y_j \in \{0,1\}
   $$


## 0 - Instalação das biblotecas

In [1]:
# Comando para instalar a biblioteca Pyomo
# Pyomo é uma biblioteca de modelagem matemática amplamente utilizada para otimização
# O sinal '-q' é usado para realizar a instalação de forma silenciosa, reduzindo a saída de log
!pip install -q pyomo

# Este comando instala o solver COIN-OR Branch and Cut (CBC) no ambiente.
# O CBC é um solver de programação linear e inteira open-source, utilizado por várias bibliotecas de otimização, incluindo Pyomo.
!apt install -y -qq coinor-cbc

# Comando para instalar o solver GLPK
# GLPK (GNU Linear Programming Kit) é um solver de programação linear que Pyomo pode utilizar
# O parâmetro '-y' confirma automaticamente a instalação, e '-qq' reduz a verbosidade do processo
!apt-get install -y -qq glpk-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.9 MB/s eta 0:00:00
The following additional packages will be installed:
  coinor-libcbc3 coinor-libcgl1 coinor-libclp1 coinor-libcoinutils3v5 coinor-libosi1v5
The following NEW packages will be installed:
  coinor-cbc coinor-libcbc3 coinor-libcgl1 coinor-libclp1 coinor-libcoinutils3v5 coinor-libosi1v5
0 upgraded, 6 newly installed, 0 to remove and 24 not upgraded.
Need to get 2,908 kB of archives.
After this operation, 8,310 kB of additional disk space will be used.
Selecting previously unselected package coinor-libcoinutils3v5:amd64.
(Reading database ... 121666 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.11.4+repack1-2_amd64.deb ...
Unpacking coinor-libcoinutils3v5:amd64 (2.11.4+repack1-2) ...
Selecting previously unselected package coinor-libosi1v5:amd64.
Preparing to unpack .../1-coinor-libos

## 1 - Solução com Pyomo e solver GLPK

In [2]:
# Importa todas as funcionalidades necessárias do Pyomo
from pyomo.environ import *

# Dados do Problema
V = 5  # Número de VMs
M = 3  # Número de servidores
d = {1: 3, 2: 5, 3: 2, 4: 6, 5: 3}  # Demanda de capacidade das VMs (em unidades de capacidade)
C = {1: 10, 2: 10, 3: 10}  # Capacidade dos servidores (em unidades de capacidade)
p = {1: 4, 2: 3, 3: 2, 4: 5, 5: 1}  # Preço de cada VM (em unidades monetárias)
w = {1: 2, 2: 2, 3: 1}  # Custo energético de cada servidor (em unidades monetárias)

# Modelo
model = ConcreteModel()

# Conjuntos
model.VMs = RangeSet(V)
model.Servidores = RangeSet(M)

# Variáveis de Decisão
model.x = Var(model.VMs, model.Servidores, domain=Binary)  # Alocação de VMs em servidores
model.y = Var(model.Servidores, domain=Binary)  # Status dos servidores (ligado ou desligado)

# Função Objetivo
# Maximiza o lucro (receita das VMs - custo dos servidores)
model.obj = Objective(expr=sum(p[i] * model.x[i, j] for i in model.VMs for j in model.Servidores) -
                      sum(w[j] * model.y[j] for j in model.Servidores), sense=maximize)

# Restrições
# Restrição de capacidade: a soma das demandas das VMs em um servidor não pode exceder sua capacidade
model.capacidade = ConstraintList()
for j in model.Servidores:
    model.capacidade.add(sum(d[i] * model.x[i, j] for i in model.VMs) <= model.y[j] * C[j])

# Restrição de alocação: cada VM deve ser alocada em exatamente um servidor
model.alocacao = ConstraintList()
for i in model.VMs:
    model.alocacao.add(sum(model.x[i, j] for j in model.Servidores) == 1)

# Resolve o modelo usando um solver
solver = SolverFactory('glpk')
resultado = solver.solve(model, tee=True)

# Verifica se a solução foi bem-sucedida e imprime os resultados
if resultado.solver.status == SolverStatus.ok and resultado.solver.termination_condition == TerminationCondition.optimal:
    # Imprime o lucro total
    print(f'\nLucro Total: {model.obj():.2f} unidades monetárias')

    # Imprime a alocação das VMs nos servidores
    print('\nAlocação das VMs nos Servidores:')
    for i in model.VMs:
        for j in model.Servidores:
            if model.x[i, j]() == 1:
                print(f'  - VM {i} alocada no Servidor {j}')

    # Imprime o status dos servidores
    print('\nStatus dos Servidores:')
    for j in model.Servidores:
        status = 'Ativo' if model.y[j]() == 1 else 'Desligado'
        print(f'  - Servidor {j}: {status}')
else:
    print('\nNão foi encontrada uma solução ótima.')


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmpjft5x06v.glpk.raw --wglp /tmp/tmpzmyikeio.glpk.glp --cpxlp
 /tmp/tmp_b8abf_p.pyomo.lp
Reading problem data from '/tmp/tmp_b8abf_p.pyomo.lp'...
/tmp/tmp_b8abf_p.pyomo.lp:103: warning: lower bound of variable 'x2' redefined
/tmp/tmp_b8abf_p.pyomo.lp:103: warning: upper bound of variable 'x2' redefined
8 rows, 18 columns, 33 non-zeros
18 integer variables, all of which are binary
121 lines were read
Writing problem data to '/tmp/tmpzmyikeio.glpk.glp'...
88 lines were written
GLPK Integer Optimizer 5.0
8 rows, 18 columns, 33 non-zeros
18 integer variables, all of which are binary
Preprocessing...
8 rows, 18 columns, 33 non-zeros
18 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+01  ratio =  1.000e+01
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 8
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
8 

##2 - Solução por Busca Exaustiva

Explicação do Código:
Este código implementa uma busca exaustiva que itera por todas as combinações possíveis de alocação de VMs e status dos servidores.
Para cada combinação, ele verifica se a alocação é válida (ou seja, se não excede a capacidade de nenhum servidor ativo).
Se a alocação for válida, ele calcula o lucro (receita das VMs menos os custos dos servidores).
Finalmente, ele compara todas as combinações válidas e seleciona a que proporciona o maior lucro.
A saída é formatada para mostrar o lucro total, a alocação das VMs e o status dos servidores, similarmente ao que foi feito com a saída do solver Pyomo.

In [3]:
# Esta linha de código importa a função product do módulo itertools.
# A função product é usada para criar um iterador que gera todas as combinações
# possíveis dos elementos de várias sequências.
from itertools import product

# Dados do Problema
V = 5  # Número de VMs
M = 3  # Número de servidores
d = {1: 3, 2: 5, 3: 2, 4: 6, 5: 3}  # Demanda de capacidade das VMs
C = {1: 10, 2: 10, 3: 10}  # Capacidade dos servidores
p = {1: 4, 2: 3, 3: 2, 4: 5, 5: 1}  # Preço de cada VM
w = {1: 2, 2: 2, 3: 1}  # Custo energético de cada servidor

# Ordenando VMs por preço (do maior para o menor) e servidores por custo (do menor para o maior)
vm_order = sorted(d.keys(), key=lambda i: p[i], reverse=True)
server_order = sorted(C.keys(), key=lambda j: w[j])

# Função para calcular o lucro
def calc_profit(allocation):
    profit = 0
    server_active = {j: False for j in C.keys()}  # Inicializa o status de servidores como desligados
    for i in d.keys():
        server = allocation[i - 1]  # Ajuste para índice base 0
        server_active[server] = True  # Define o servidor como ativo
        profit += p[i]  # Adiciona o preço da VM ao lucro

    for j in C.keys():
        if server_active[j]:
            profit -= w[j]  # Subtrai o custo energético do servidor ao lucro se estiver ativo

    return profit

# Função para verificar se a alocação é válida (respeita as restrições de capacidade)
def is_valid(allocation):
    server_load = {j: 0 for j in C.keys()}  # Inicializa a carga dos servidores como zero
    for i in d.keys():
        server = allocation[i - 1]  # Ajuste para índice base 0
        server_load[server] += d[i]  # Adiciona a demanda da VM à carga do servidor
        if server_load[server] > C[server]:
            return False  # Retorna falso se a capacidade do servidor for excedida
    return True

# Busca Exaustiva
best_profit = float('-inf')
best_allocation = None
for allocation in product(range(1, M + 1), repeat=V):  # Ajuste para índice base 1
    if is_valid(allocation):
        profit = calc_profit(allocation)
        if profit > best_profit:
            best_profit = profit
            best_allocation = allocation

# Resultados
best_server_status = [1 if allocation[i] in server_order else 0 for i in range(V)]  # Verifica o status de cada servidor

# Exibindo a melhor solução
print(f'Lucro Total: {best_profit:.2f} unidades monetárias')

print('\nAlocação das VMs nos Servidores:')
for i in range(V):
    for j in range(M):
        if best_allocation[i] == server_order[j]:
            print(f'  - VM {i+1} alocada no Servidor {server_order[j]}')

print('\nStatus dos Servidores:')
for j in range(M):
    status = 'Ativo' if best_server_status[j] == 1 else 'Desligado'
    print(f'  - Servidor {server_order[j]}: {status}')

Lucro Total: 12.00 unidades monetárias

Alocação das VMs nos Servidores:
  - VM 1 alocada no Servidor 1
  - VM 2 alocada no Servidor 1
  - VM 3 alocada no Servidor 1
  - VM 4 alocada no Servidor 3
  - VM 5 alocada no Servidor 3

Status dos Servidores:
  - Servidor 3: Ativo
  - Servidor 1: Ativo
  - Servidor 2: Ativo


##3 - Comparação das soluções

## Código 1: Otimização com Pyomo

### Objetivo
- Maximizar o lucro através do preço das VMs e custo dos servidores.

### Método
- Utiliza Pyomo e o solver GLPK para definir um modelo de programação linear.

### Variáveis de Decisão
- `model.x`: Alocação de VMs em servidores (binária).
- `model.y`: Status dos servidores (ligado ou desligado, binário).

### Restrições
- Capacidade dos servidores e alocação de VMs.

### Solver
- Usa GLPK para encontrar a solução ótima.

### Resultados
- Alocação ótima de VMs e lucro total.

---

## Código 2: Busca Exaustiva

### Objetivo
- Maximizar o lucro através do preço das VMs e custo dos servidores.

### Método
- Abordagem de busca exaustiva para testar todas as alocações possíveis.

### Validação
- Verifica a validade da alocação baseada na capacidade dos servidores.

### Lucro
- Calculado pela soma dos preços das VMs menos os custos dos servidores ativos.

### Resultados
- Melhor alocação de VMs e lucro total.

---

# Comparação

- **Eficiência**: O código de Pyomo é mais eficiente e escalável, enquanto a busca exaustiva pode ser ineficiente para grandes instâncias.
- **Complexidade**: O Pyomo adiciona uma camada de complexidade, mas oferece uma estrutura mais poderosa para otimização.
- **Flexibilidade**: O modelo de Pyomo é mais adaptável a mudanças e expansões.
- **Dependência de Ferramentas Externas**: O primeiro código depende de Pyomo e GLPK, enquanto o segundo é autossuficiente em Python.

# Conclusão

Ambos resolvem o mesmo problema, mas com abordagens diferentes. O primeiro é mais robusto e escalável, ideal para instâncias maiores e com requisitos de otimização mais complexos. O segundo é mais simples, mas menos eficiente para grandes problemas. Notou-se que na primeira abordagem, apenas os Servidores 1 e 3 foram ativos e alocados com VMs, enquanto na abordagem exaustiva, os três Servidores foram ativos, no entanto, apenas os Servidores 1 e 3 foram alocados com VMs.
